In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("River-Unity/summary_statistics.csv")
df

,Unnamed: 0,order,group,centerline_x,centerline_y,centerline_x_corrected,centerline_y_corrected,right_bank_x,right_bank_y,left_bank_x,left_bank_y,width (m),curvature
0,0,1,1,0.270000,0.026172,0.270000,0.026172,0.221250,0.000000,0.318750,0.052344,0.110662,0
1,1,2,1,0.261469,0.035500,0.261469,0.035500,0.214688,0.010531,0.308250,0.060469,0.106055,0
2,2,3,1,0.255500,0.046281,0.255500,0.046281,0.208063,0.021281,0.302938,0.071281,0.107244,0
3,3,4,1,0.249719,0.057156,0.249719,0.057156,0.201563,0.032094,0.297875,0.082219,0.108575,0
4,4,5,1,0.244094,0.068172,0.244094,0.068172,0.195156,0.043031,0.293031,0.093313,0.110035,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20665,20665,20666,1,76.349110,80.804880,76.349110,80.804880,76.289850,80.825910,76.408380,80.783840,0.125774,0
20666,20666,20667,1,76.352250,80.817310,76.352250,80.817310,76.293280,80.838500,76.411220,80.796130,0.125324,0
20667,20667,20668,1,76.354600,80.830060,76.354600,80.830060,76.295630,80.851500,76.413570,80.808630,0.125485,0
20668,20668,20669,1,76.356020,80.843220,76.356020,80.843220,76.296690,80.865040,76.415340,80.821410,0.126418,0
